# Merging LandCover.ai and DeepGlobe dataset

### todo
- [ ] find fitting resolution
- [ ] transfer labels to same
- [ ] Resize resolution
- [ ] v0 verwenden weil da keine straßen als label sind??

### additional info

using landcover.ai v1

In [32]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [33]:
# data root
LANDCOVER_AI_ROOT = '../data/landcover.ai.v1'
DEEPGLOBE_ROOT = '../data/deepglobe'
MERGED_ROOT = '../data/merge'

Landcover.ai dataset masks need to be mapped from class to color to visualise the pictures 

In [38]:
from notebooks.utils import split_images
# cropping images due to gpu resource and the need of fixed size for the algorithm
split_images(LANDCOVER_AI_ROOT)
split_images(DEEPGLOBE_ROOT + '/train')
split_images(FIRST_LANDCOVER_AI_ROOT + '/train')

In [39]:
FIRST_LANDCOVER_AI_ROOT = '../data/landcover.ai.v0'
split_images(FIRST_LANDCOVER_AI_ROOT)


The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exists.
The image-file ../data/merge\M-33-20-D-c-4-2_0.jpg already exi

In [ ]:
# pytorsch
# batch normalisier
# normalisieren wertebereich
# euqalizer nur schwarz weiß
# noise reducting min max 
# blurr
# one hot
# yolo vs cnn
#

urban_land,0,255,255
agriculture_land,255,255,0
rangeland,255,0,255
forest_land,0,255,0
water,0,0,255
barren_land,255,255,255
unknown,0,0,0

building (1), woodland (2), water(3), road(4)
->
building 1 -> 0,255,255
woodland 2 -> 0,255,0
water    3 -> 0,0,255
road     4 -> 0,0,0


In [43]:
import torch
from torch.utils.data import DataLoader
from time import time
import multiprocessing as mp
for num_workers in range(2, mp.cpu_count(), 2):
    train_loader = DataLoader(train_reader,shuffle=True,num_workers=num_workers,batch_size=64,pin_memory=True)
start = time()
for epoch in range(1, 3):
    for i, data in enumerate(train_loader, 0):
        pass
end = time()
print("Finish with:{} second, num_workers={}".format(end - start, num_workers))

NameError: name 'train_reader' is not defined